In [205]:
#import dependencies
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import requests
import datetime
from scipy.stats import linregress

#known issues
## zipcode "Texas" to 75042
## 7038 zip needs to be 75038

In [206]:
#file path
file = "combinedmlsdata.csv"

#read the file into pandas
df = pd.read_csv(file, sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170566 entries, 0 to 170565
Data columns (total 32 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   MLS #                            170566 non-null  object
 1   Unnamed: 1                       170566 non-null  object
 2   Address                          170566 non-null  object
 3   City                             170566 non-null  object
 4   List Price                       170566 non-null  object
 5   Original List Price              170549 non-null  object
 6   SqFt Total                       170557 non-null  object
 7   Close Price                      133254 non-null  object
 8   RATIO Current Price By SQFT      170507 non-null  object
 9   Number Of Stories                170565 non-null  object
 10  Beds Total                       170566 non-null  object
 11  Baths Total                      170566 non-null  object
 12  Parking Spaces G

In [207]:
df.columns.values.tolist()

['MLS #',
 'Unnamed: 1',
 'Address',
 'City',
 'List Price',
 'Original List Price',
 'SqFt Total',
 'Close Price',
 'RATIO Current Price By SQFT',
 'Number Of Stories',
 'Beds Total',
 'Baths Total',
 'Parking Spaces Garage',
 'Lot Size Area',
 'Yr Built',
 'Pool',
 'DOM',
 'Listing Date',
 'Close Date',
 'Expiration Date',
 'Cancelled Date',
 'Status Change Timestamp',
 'List Agent Full Name',
 'Buyers/SubAgent Full Name',
 'HOA',
 'MLS SubArea',
 'Property Sub Type',
 'Property Type',
 'RATIO Close Price By List Price',
 'School District',
 'Subdivision Name',
 'Zip Code']

In [208]:
#rename the unnamed column to Status for transaction status
mls_df = df.rename(columns={"Unnamed: 1": "Status"})
mls_df.dtypes

MLS #                              object
Status                             object
Address                            object
City                               object
List Price                         object
Original List Price                object
SqFt Total                         object
Close Price                        object
RATIO Current Price By SQFT        object
Number Of Stories                  object
Beds Total                         object
Baths Total                        object
Parking Spaces Garage              object
Lot Size Area                      object
Yr Built                           object
Pool                               object
DOM                                object
Listing Date                       object
Close Date                         object
Expiration Date                    object
Cancelled Date                     object
Status Change Timestamp            object
List Agent Full Name               object
Buyers/SubAgent Full Name         

In [209]:
mlsconvert_df = mls_df.convert_dtypes()

mlsconvert_df.dtypes

MLS #                              string
Status                             string
Address                            string
City                               string
List Price                         string
Original List Price                string
SqFt Total                         string
Close Price                        string
RATIO Current Price By SQFT        string
Number Of Stories                  string
Beds Total                         string
Baths Total                        string
Parking Spaces Garage              string
Lot Size Area                      string
Yr Built                           string
Pool                               string
DOM                                string
Listing Date                       string
Close Date                         string
Expiration Date                    string
Cancelled Date                     string
Status Change Timestamp            string
List Agent Full Name               string
Buyers/SubAgent Full Name         

In [210]:
# Replace remaining NaN fields across the data set with UnKnown
mls = mlsconvert_df.replace(to_replace = np.nan, value = '')

In [211]:
mls.dtypes

MLS #                              string
Status                             string
Address                            string
City                               string
List Price                         string
Original List Price                string
SqFt Total                         string
Close Price                        string
RATIO Current Price By SQFT        string
Number Of Stories                  string
Beds Total                         string
Baths Total                        string
Parking Spaces Garage              string
Lot Size Area                      string
Yr Built                           string
Pool                               string
DOM                                string
Listing Date                       string
Close Date                         string
Expiration Date                    string
Cancelled Date                     string
Status Change Timestamp            string
List Agent Full Name               string
Buyers/SubAgent Full Name         

In [212]:
mls.isnull().sum()


MLS #                              0
Status                             0
Address                            0
City                               0
List Price                         0
Original List Price                0
SqFt Total                         0
Close Price                        0
RATIO Current Price By SQFT        0
Number Of Stories                  0
Beds Total                         0
Baths Total                        0
Parking Spaces Garage              0
Lot Size Area                      0
Yr Built                           0
Pool                               0
DOM                                0
Listing Date                       0
Close Date                         0
Expiration Date                    0
Cancelled Date                     0
Status Change Timestamp            0
List Agent Full Name               0
Buyers/SubAgent Full Name          0
HOA                                0
MLS SubArea                        0
Property Sub Type                  0
P

In [217]:
#clean up the data types

#convert all number data types to float or integer
mls_df[["MLS #"]] = mls_df[["MLS #"]].apply(pd.to_numeric)
mls_df[["List Price"]] = mls_df[["List Price"]].astype(float)
mls_df[["Original List Price"]] = mls_df[["Original List Price"]].astype(float)

mls_df[["SqFt Total", "Close Price", "Number Of Stories", "RATIO Current Price By SQFT", 
        "Beds Total", "Baths Total", "Parking Spaces Garage", "Lot Size Area", "DOM", 
        "RATIO Close Price By List Price", "Zip Code"]] \
= mls_df[["SqFt Total", "Close Price", "Number Of Stories", "RATIO Current Price By SQFT", 
          "Beds Total", "Baths Total", "Parking Spaces Garage", "Lot Size Area", "DOM", 
          "RATIO Close Price By List Price", "Zip Code"]].apply(pd.to_numeric)

mls_df[["Yr Built"]] = mls_df[["Yr Built"]].astype(float)

#convert all dates to date
mls_df[["Listing Date"]] = pd.to_datetime(mls_df["Listing Date"], format='%m/%d/%y')
mls_df[["Close Date"]] = pd.to_datetime(mls_df["Close Date"], format='%m/%d/%y')
mls_df[["Expiration Date"]] = pd.to_datetime(mls_df["Expiration Date"], format='%m/%d/%y')
mls_df[["Cancelled Date"]] = pd.to_datetime(mls_df["Cancelled Date"], format='%m/%d/%y')
mls_df[["Status Change Timestamp"]] = pd.to_datetime(mls_df["Status Change Timestamp"], format='%m/%d/%y')

#convert all strings from objects to string

mls_df[['Status', 'Address', 'City', 'List Agent Full Name', 'Buyers/SubAgent Full Name', 
        'MLS SubArea', 'Property Sub Type', 'Property Type', 'School District', 'Subdivision Name', 'HOA', 'Pool']] \
= mls_df[['Status', 'Address', 'City', 'List Agent Full Name', 'Buyers/SubAgent Full Name', 
        'MLS SubArea', 'Property Sub Type', 'Property Type', 'School District', 'Subdivision Name', 'HOA', 'Pool']].convert_dtypes()
    
mls_df['Status'] = mls_df['Status'].convert_dtypes()

mls_df.dtypes

MLS #                                       int64
Status                                     string
Address                                    string
City                                       string
List Price                                float64
Original List Price                       float64
SqFt Total                                float64
Close Price                               float64
RATIO Current Price By SQFT               float64
Number Of Stories                         float64
Beds Total                                  int64
Baths Total                               float64
Parking Spaces Garage                     float64
Lot Size Area                             float64
Yr Built                                  float64
Pool                                       string
DOM                                         int64
Listing Date                       datetime64[ns]
Close Date                         datetime64[ns]
Expiration Date                    datetime64[ns]


In [214]:
mls_df.describe()

,MLS #,List Price,Original List Price,SqFt Total,Close Price,RATIO Current Price By SQFT,Number Of Stories,Beds Total,Baths Total,Parking Spaces Garage,Lot Size Area,Yr Built,DOM,RATIO Close Price By List Price,Zip Code
count,1.705660e+05,1.705660e+05,1.705490e+05,170557.000000,1.332540e+05,170507.000000,170565.000000,170566.000000,170566.00000,170469.000000,169790.000000,170561.000000,170566.000000,133254.000000,170566.000000
mean,1.390094e+07,4.174742e+05,4.324985e+05,2180.710484,3.544179e+05,167.849893,1.445349,3.226194,2.27437,1.591891,3.088491,1981.131466,49.459576,0.985307,75155.908464
std,3.570198e+05,7.765706e+05,1.246974e+06,1561.442562,4.328195e+05,111.566325,0.617541,0.930758,0.90486,0.920248,161.572103,68.738415,67.759028,0.085270,183.672527
min,9.108100e+06,0.000000e+00,0.000000e+00,0.000000,1.000000e+01,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,1800.000000,-477.000000,0.000070,7038.000000
25%,1.360218e+07,1.799000e+05,1.805000e+05,1416.000000,1.730000e+05,108.620000,1.000000,3.000000,2.00000,1.000000,0.160000,1962.000000,6.000000,0.964910,75080.000000
50%,1.390759e+07,2.650000e+05,2.699000e+05,1882.000000,2.450000e+05,141.430000,1.000000,3.000000,2.00000,2.000000,0.193000,1981.000000,24.000000,0.990550,75204.000000
75%,1.419954e+07,4.350000e+05,4.490000e+05,2582.000000,3.890000e+05,202.290000,2.000000,4.000000,3.00000,2.000000,0.270000,2001.000000,68.000000,1.000530,75225.000000
max,1.455680e+07,1.899000e+08,3.418200e+08,398618.000000,3.920000e+07,22317.550000,9.000000,10.000000,17.50000,15.000000,29325.000000,9999.000000,3437.000000,10.350520,76752.000000
